In [ ]:
import os
import pickle
import numpy
from music21 import note, chord

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow.keras as keras
from models import get_distinct, create_lookups, prepare_sequences, get_music_list, create_network

In [ ]:
# run params
section = 'compose'
run_id = '0001'
music_name = 'cello'
run_folder = 'run/{}/'.format(section)
run_folder += '_'.join([run_id, music_name])

store_folder = os.path.join(run_folder, 'store')
data_folder = os.path.join('data', music_name)

if not os.path.exists(run_folder):
    os.makedirs(run_folder)
    os.makedirs(os.path.join(run_folder, 'store'))
    os.makedirs(os.path.join(run_folder, 'output'))
    os.makedirs(os.path.join(run_folder, 'weights'))
    os.makedirs(os.path.join(run_folder, 'viz'))

mode = 'build'  # 'load'

# data params
intervals = range(1)
seq_len = 32

# model params
embed_size = 100
rnn_units = 256
use_attention = True


In [ ]:
if mode == 'build':
    music_list, parser = get_music_list(data_folder)
    print(len(music_list), 'files in total')

    notes = []
    durations = []

    for i, file in enumerate(music_list):
        print(i+1, "Parsing %s" % file)
        original_score = parser.parse(file).chordify()

        for interval in intervals:
            score = original_score.transpose(interval)

            notes.extend(['START'] * seq_len)
            durations.extend([0] * seq_len)

            for element in score.flat:
                if isinstance(element, note.Note):
                    if element.isRest:
                        notes.append(str(element.name))
                        durations.append(element.duration.quarterLength)
                    else:
                        notes.append(str(element.nameWithOctave))
                        durations.append(element.duration.quarterLength)

                if isinstance(element, chord.Chord):
                    notes.append(
                        '.'.join(n.nameWithOctave for n in element.pitches))
                    durations.append(element.duration.quarterLength)

    with open(os.path.join(store_folder, 'notes'), 'wb') as f:
        # ['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
        pickle.dump(notes, f)
    with open(os.path.join(store_folder, 'durations'), 'wb') as f:
        pickle.dump(durations, f)
else:
    with open(os.path.join(store_folder, 'notes'), 'rb') as f:
        # ['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
        notes = pickle.load(f)
    with open(os.path.join(store_folder, 'durations'), 'rb') as f:
        durations = pickle.load(f)

In [ ]:
# get the distinct sets of notes and durations
note_names, n_notes = get_distinct(notes)
duration_names, n_durations = get_distinct(durations)
distincts = [note_names, n_notes, duration_names, n_durations]

with open(os.path.join(store_folder, 'distincts'), 'wb') as f:
    pickle.dump(distincts, f)

# make the lookup dictionaries for notes and dictionaries and save
note_to_int, int_to_note = create_lookups(note_names)
duration_to_int, int_to_duration = create_lookups(duration_names)
lookups = [note_to_int, int_to_note, duration_to_int, int_to_duration]

with open(os.path.join(store_folder, 'lookups'), 'wb') as f:
    pickle.dump(lookups, f)

print('\nnote_to_int')
note_to_int
print('\nduration_to_int')
duration_to_int

In [ ]:
network_input, network_output = prepare_sequences(notes, durations, lookups, distincts, seq_len)

print('pitch input')
print(network_input[0][0])
print('duration input')
print(network_input[1][0])
print('pitch output')
print(network_output[0][0])
print('duration output')
print(network_output[1][0])

In [ ]:
model, att_model = create_network(
    n_notes, n_durations, embed_size, rnn_units, use_attention)
keras.utils.plot_model(model, to_file=os.path.join(
    run_folder, 'viz/model.png'), show_shapes=True, show_layer_names=True)
model.summary()


In [ ]:
weights_folder = os.path.join(run_folder, 'weights')

checkpoint1 = keras.callbacks.ModelCheckpoint(
    os.path.join(weights_folder,
                 "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

checkpoint2 = keras.callbacks.ModelCheckpoint(
    os.path.join(weights_folder, "weights.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

early_stopping = keras.callbacks.EarlyStopping(
    monitor='loss', restore_best_weights=True, patience=10
)


callbacks_list = [
    checkpoint1, checkpoint2, early_stopping
]

model.save_weights(os.path.join(weights_folder, "weights.h5"))
model.fit(network_input, network_output, epochs=2000000, batch_size=32, validation_split=0.2, callbacks=callbacks_list, shuffle=True)